1. Install AzureML-SDK
2. Import dependencies (Workspace, Model, Environment, CondaDependencies, InferenceConfig, AciWebservice, Webservice)
3. Create a Resource Group from Azure Portal
4. Register Model in Azure ML
5. Create a Conda environment for my scikit-learn model
6. Create an Inference Configuration
7. Specify deplloyment configuration for ACI
8. Deploy and Test model

Note:
- ACI (Azure container instances): a service enables to deploy (Linux and Windows) containers on Azure public cloud without having to provision or manage any underlying infrastructure.
- AciWebservice: 
    - represents a ML model deployed as a web service endpoint on Azure Container Instances
    - a deployed service created from a model, script, and associated files. 
    - Output: a load-balanced, HTTP endpoint with a REST API
        - can send data to this API and receive the prediction returned by the model
- Inference Configuration: 
    - an input parameter for Model deployment-related actions (deploy, profile, package)
    - represents configuration settings for a custom environment used for deployment

In [ ]:
#!pip install azureml-sdk
# quiet without displaying any output <=> -q
!pip install -q azureml-sdk 

In [ ]:
import os
import json
import requests

from azureml.core import Workspace
from azureml.core.model import Model
from azureml.core.environment import Environment
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice, Webservice

In [ ]:
## Loading the configuration file
# use 
#  1) .env 
#  2) load_dotenv files from python-dotenv module

config_file_path = "config.json"

# Read JSON data into a dictionary
with open(config_file_path, 'r') as file:
    data = json.load(file)
    
subscription_id = data["subscription_id"]
resource_group = data["resource_group"]
workspace_name = data["workspace_name"]
region = data["region"]

In [ ]:
print(resource_group)
print(workspace_name)
print(region)

#### Step 1: create a Resource Group from Azure Portal

In [ ]:
## Create a workspace
ws = Workspace.create(
                        name=workspace_name,
                        subscription_id=subscription_id,
                        resource_group=resource_group,
                        location=region
)

In [ ]:
print(f'Workspace {workspace_name} is created')

In [ ]:
# specify the path to your model file
model_path = 'diabetes_model.pkl'
model_name = 'diabetes_prediction_model'

#### Step 2: register the model in Azure ML

In [ ]:
registered_model = Model.register(
                                    model_path=model_path,
                                    model_name=model_name,
                                    workspace=ws
)

#### Step 3: create a Conda environment for my scikit-learn model

In [ ]:
conda_env = Environment('my-conda-env')
conda_env.python.conda_dependencies = CondaDependencies.create(
    conda_packages=['scikit-learn']
)

#### Step 4: create an Inference Configuration

In [ ]:
inference_config = InferenceConfig(
    entry_script='score.py', environment=conda_env
)

#### Step 5: specify deployment configuration for ACI

In [ ]:
aci_config = AciWebservice.deploy_configuration(
    cpu_cores=1, memory_gb=1
)

In [ ]:
service = Model.deploy(
    workspace=ws,
    name='diabetes-prediction-service',
    models=[registered_model],
    inference_config=inference_config,
    deployment_config=aci_config
)
service.wait_for_deployment(show_output=True)

In [ ]:
scoring_uri = service.scoring_uri
scoring_uri

### Cleaning up all the created resources

In [ ]:
# Delete the service
service_name = 'diabetes-prediction-service'

service = ws.webservices[service_name]
service.delete()

# Delete the model
model_name = 'diabetes_prediction_model'
registered_model = ws.models[model_name]
registered_model.delete()

# Delete the workspace
ws.delete(delete_dependent_resources=True)